In [100]:
import pandas as pd
import pprint
import subprocess
import os
import datetime
import pandas as pd
from tqdm import tqdm
import time
import requests
import os
import re
import logging
from PIL import Image
import csv
from io import BytesIO
from time import sleep
from typing import Any, BinaryIO, Dict, Iterable, List, Optional, Tuple

from pyinaturalist import get_observations

In [107]:
import kagglehub

# Download the latest version of the dataset
path = kagglehub.dataset_download("iamsouravbanerjee/animal-image-dataset-90-different-animals")

print("Path to dataset files:", path)


100%|██████████| 656M/656M [02:36<00:00, 4.41MB/s] 

Extracting files...


Path to dataset files: /Users/nandapop/.cache/kagglehub/datasets/iamsouravbanerjee/animal-image-dataset-90-different-animals/versions/5


In [13]:
observations_file = 'observations.csv'
photos_file = 'photos.csv'
taxa_file = 'taxa.csv'


In [14]:
taxa_samples = pd.read_csv('/Volumes/Ramirez/inaturalist-open-data/taxa.csv',sep='\t',nrows=5)
taxa_samples

,taxon_id,ancestry,rank_level,rank,name,active
0,8943,48460/1/2/355675/3/7251/980017,20,genus,Calyptura,True
1,5270,48460/1/2/355675/3/71261/5067/5269,10,species,Ichthyophaga ichthyaetus,False
2,2031,48460/1/2/355675/3/573/2017/2030,10,species,Talegalla jobiensis,True
3,27,48460/1/2/355675/3/4/23/24,10,species,Grus canadensis,False
4,8606,48460/1/2/355675/3/7251/71348/8605,10,species,Lophorina superba,False


In [28]:
observations_sample = pd.read_csv('/Volumes/Ramirez/inaturalist-open-data/observations.csv',sep='\t',nrows=5)
print(observations_sample.columns)
observations_sample.info()

Index(['observation_uuid', 'observer_id', 'latitude', 'longitude',
       'positional_accuracy', 'taxon_id', 'quality_grade', 'observed_on'],
      dtype='object')
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   observation_uuid     5 non-null      object 
 1   observer_id          5 non-null      int64  
 2   latitude             5 non-null      float64
 3   longitude            5 non-null      float64
 4   positional_accuracy  0 non-null      float64
 5   taxon_id             5 non-null      int64  
 6   quality_grade        5 non-null      object 
 7   observed_on          5 non-null      object 
dtypes: float64(3), int64(2), object(3)
memory usage: 448.0+ bytes


In [67]:
observations_cols = ['observation_uuid', 'latitude','longitude', 'taxon_id', 'observed_on']
observations_file = '/Volumes/Ramirez/inaturalist-open-data/observations.csv'
#output_file = '/Users/nandapop/Documents/Bootcamp/iNaturalist/data/raw/processed_observations.csv'
output_file = '/Volumes/Ramirez/inaturalist-open-data/processed/processed_observations2.csv'

In [68]:
observations_chunks = pd.read_csv(
    observations_file,
    sep='\t',
    usecols=observations_cols,
    chunksize= 70000
)

is_first_chunk = True

for chunk in observations_chunks:
  
    chunk['processed'] = True
    
    # Save the chunk to CSV
    if is_first_chunk:
        # Write the header for the first chunk
        chunk.to_csv(output_file, index=False)
        is_first_chunk = False
    else:
        # Append to the CSV file without writing the header
        chunk.to_csv(output_file, mode='a', header=False, index=False)


In [55]:
chunk_observations= pd.read_csv('data/raw/processed_observations.csv')
chunk_observations.head()

,observation_uuid,latitude,longitude,taxon_id,observed_on,processed
0,7ae155fc-f49e-4e4f-91c5-51e31e805478,32.189935,-80.758484,203485.0,2009-10-31,True
1,05baefa2-028c-4c6a-b459-37f7161ca374,33.430609,-111.027062,153887.0,2009-08-15,True
2,5c63a4aa-9828-4f6f-b181-7e71b4af24fa,33.404743,-111.937348,313499.0,2009-08-22,True
3,e9bfdce3-556c-4506-890b-9e65d241a929,33.430609,-111.027062,790491.0,2009-08-15,True
4,19ad8eca-938a-4031-ba75-edee2749b247,32.216316,-80.752608,48505.0,2007-10-31,True


In [76]:
chunk_observations2 = pd.read_csv('/Volumes/Ramirez/inaturalist-open-data/observations.csv',sep='\t',nrows=70000)
chunk_observations2.head()


,observation_uuid,observer_id,latitude,longitude,positional_accuracy,taxon_id,quality_grade,observed_on
0,7ae155fc-f49e-4e4f-91c5-51e31e805478,516,32.189935,-80.758484,NaN,203485.0,research,2009-10-31
1,05baefa2-028c-4c6a-b459-37f7161ca374,516,33.430609,-111.027062,NaN,153887.0,research,2009-08-15
2,5c63a4aa-9828-4f6f-b181-7e71b4af24fa,516,33.404743,-111.937348,NaN,313499.0,research,2009-08-22
3,e9bfdce3-556c-4506-890b-9e65d241a929,516,33.430609,-111.027062,NaN,790491.0,research,2009-08-15
4,19ad8eca-938a-4031-ba75-edee2749b247,516,32.216316,-80.752608,NaN,48505.0,research,2007-10-31


In [77]:
chunk_observations2.isnull().sum()


observation_uuid           0
observer_id                0
latitude                 888
longitude                888
positional_accuracy    37594
taxon_id                  20
quality_grade              0
observed_on              338
dtype: int64

In [78]:
chunk_observations2 = chunk_observations2.dropna()

In [79]:
chunk_observations2.isnull().sum()


observation_uuid       0
observer_id            0
latitude               0
longitude              0
positional_accuracy    0
taxon_id               0
quality_grade          0
observed_on            0
dtype: int64

In [80]:
chunk_observations2.duplicated().sum()

0

In [2]:
photos_sample = pd.read_csv('/Volumes/Ramirez/inaturalist-open-data/photos.csv',sep='\t',nrows=5)
print(photos_sample.columns)
photos_sample.info()
photos_sample.head()

Index(['photo_uuid', 'photo_id', 'observation_uuid', 'observer_id',
       'extension', 'license', 'width', 'height', 'position'],
      dtype='object')
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   photo_uuid        5 non-null      object
 1   photo_id          5 non-null      int64 
 2   observation_uuid  5 non-null      object
 3   observer_id       5 non-null      int64 
 4   extension         5 non-null      object
 5   license           5 non-null      object
 6   width             5 non-null      int64 
 7   height            5 non-null      int64 
 8   position          5 non-null      int64 
dtypes: int64(5), object(4)
memory usage: 488.0+ bytes


,photo_uuid,photo_id,observation_uuid,observer_id,extension,license,width,height,position
0,8d6b2534-d30a-47a8-bc1c-986a21817997,21213,7ae155fc-f49e-4e4f-91c5-51e31e805478,516,jpg,CC-BY-NC,584,389,0
1,6e8112fd-f703-4052-94da-b7cfc03ff3d4,21216,7ae155fc-f49e-4e4f-91c5-51e31e805478,516,jpg,CC-BY-NC,584,389,1
2,49141c2f-48b0-4671-9cee-fce6efd11822,21215,05baefa2-028c-4c6a-b459-37f7161ca374,516,jpg,CC-BY-NC,584,389,0
3,71090faa-9110-4df7-bb8f-af2415fe2e72,21214,5c63a4aa-9828-4f6f-b181-7e71b4af24fa,516,jpg,CC-BY-NC,584,389,0
4,92c703d0-20f1-4da9-af9c-2a6bdb6db53b,21217,e9bfdce3-556c-4506-890b-9e65d241a929,516,jpg,CC-BY-NC,584,389,0


In [42]:
photos_cols = ['photo_uuid', 'photo_id','observation_uuid', 'extension']
photos_file = '/Volumes/Ramirez/inaturalist-open-data/photos.csv'
output_file = '/Users/nandapop/Documents/Bootcamp/iNaturalist/data/raw/processed_photos.csv'

In [5]:
observations_chunks = pd.read_csv(
    photos_file,
    sep='\t',
    usecols=photos_cols,
    chunksize= 40000
)

is_first_chunk = True

for chunk in observations_chunks:
  
    chunk['processed'] = True
    
    # Save the chunk to CSV
    if is_first_chunk:
        # Write the header for the first chunk
        chunk.to_csv(output_file, index=False)
        is_first_chunk = False
    else:
        # Append to the CSV file without writing the header
        chunk.to_csv(output_file, mode='a', header=False, index=False)


In [43]:
processed_photos = pd.read_csv('/Volumes/Ramirez/inaturalist-open-data/processed/photos.csv',nrows=40000)
processed_photos.head()

,photo_uuid,photo_id,observation_uuid,extension,processed
0,8d6b2534-d30a-47a8-bc1c-986a21817997,21213,7ae155fc-f49e-4e4f-91c5-51e31e805478,jpg,True
1,6e8112fd-f703-4052-94da-b7cfc03ff3d4,21216,7ae155fc-f49e-4e4f-91c5-51e31e805478,jpg,True
2,49141c2f-48b0-4671-9cee-fce6efd11822,21215,05baefa2-028c-4c6a-b459-37f7161ca374,jpg,True
3,71090faa-9110-4df7-bb8f-af2415fe2e72,21214,5c63a4aa-9828-4f6f-b181-7e71b4af24fa,jpg,True
4,92c703d0-20f1-4da9-af9c-2a6bdb6db53b,21217,e9bfdce3-556c-4506-890b-9e65d241a929,jpg,True


In [11]:
processed_photos.isnull().sum()


photo_uuid          0
photo_id            0
observation_uuid    0
extension           0
processed           0
dtype: int64

In [27]:
processed_photos.dtypes


photo_uuid          object
photo_id             int64
observation_uuid    object
extension           object
processed             bool
dtype: object

The photo_id and extension can be used to fetch the appropriate photo files from among the tens of millions in the iNaturalist Open Dataset. The file path in the S3 bucket is s3://inaturalist-open-data/photos/[photo_id]/medium.[extension], and the file URL is https://inaturalist-open-data.s3.amazonaws.com/photos/[photo_id]/medium.[extension].

https://inaturalist-open-data.s3.amazonaws.com/photos/21213/medium.jpg
s3://inaturalist-open-data/photos/21213/medium.jpg
s3://inaturalist-open-data/photos/[photo_id]/medium.[extension]

original - 2048px
large - 1024px
medium - 500px
small - 240px
thumb - 100px
square - exactly 75x75px, cropped to be square

In [36]:
photo_id = processed_photos['photo_id'].iloc[0]
print(photo_id)
ext = processed_photos['extension'].iloc[0]
ext = processed_photos['extension'].astype('string').iloc[0]
print("ext",ext)
link = 's3://inaturalist-open-data/photos/'+str(photo_id)+'/medium'+'.'+ext
print(link)



21213
ext jpg
s3://inaturalist-open-data/photos/21213/medium.jpg


In [38]:
taxon = pd.read_csv('/Users/nandapop/Documents/Bootcamp/iNaturalist/data/raw/taxa.csv', sep='\t')
taxon.head()

,taxon_id,ancestry,rank_level,rank,name,active
0,8943,48460/1/2/355675/3/7251/980017,20.0,genus,Calyptura,True
1,5270,48460/1/2/355675/3/71261/5067/5269,10.0,species,Ichthyophaga ichthyaetus,False
2,2031,48460/1/2/355675/3/573/2017/2030,10.0,species,Talegalla jobiensis,True
3,27,48460/1/2/355675/3/4/23/24,10.0,species,Grus canadensis,False
4,8606,48460/1/2/355675/3/7251/71348/8605,10.0,species,Lophorina superba,False


In [39]:
taxon.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1534935 entries, 0 to 1534934
Data columns (total 6 columns):
 #   Column      Non-Null Count    Dtype  
---  ------      --------------    -----  
 0   taxon_id    1534935 non-null  int64  
 1   ancestry    1523077 non-null  object 
 2   rank_level  1534903 non-null  float64
 3   rank        1534935 non-null  object 
 4   name        1534935 non-null  object 
 5   active      1534935 non-null  bool   
dtypes: bool(1), float64(1), int64(1), object(3)
memory usage: 60.0+ MB


In [61]:
taxon.isnull().sum()


taxon_id          0
ancestry      11858
rank_level       32
rank              0
name              0
active            0
dtype: int64

In [62]:
taxon = taxon.dropna()

In [81]:
obs_taxon = pd.merge(chunk_observations2, taxon, on = "taxon_id")
obs_taxon.to_csv("/Users/nandapop/Documents/Bootcamp/iNaturalist/data/processed/obs_taxon.csv")

In [82]:
obs_taxon.head()

,observation_uuid,observer_id,latitude,longitude,positional_accuracy,taxon_id,quality_grade,observed_on,ancestry,rank_level,rank,name,active
0,b351c847-d2c6-412d-8d17-a58158893a31,516,32.767629,-117.233902,29066.0,41478.0,research,2011-01-16,48460/1/2/355675/40151/848317/848320/848324/15...,10.0,species,Eschrichtius robustus,True
1,8ad4179c-6e49-49c4-9a99-2f3019ddb573,516,34.404714,-119.692740,28818.0,4209.0,research,2008-02-12,48460/1/2/355675/3/67563/4203/4204,10.0,species,Podiceps auritus,True
2,79a60b82-6f37-4eb2-b17c-4a628f9314af,516,-43.803666,172.968262,27411.0,4848.0,research,2007-06-17,48460/1/2/355675/3/67561/71325/4837,10.0,species,Haematopus unicolor,True
3,c117a8f1-e73e-4cc9-8c34-36ff4441b72b,516,-43.942181,172.875589,27411.0,204520.0,research,2007-06-17,48460/1/2/355675/3/2708/2715/3484,10.0,species,Hemiphaga novaeseelandiae,True
4,4f95b972-dd7f-47af-babd-02ce30b3eda0,516,32.914267,-110.733779,29039.0,12024.0,research,2008-10-11,48460/1/2/355675/3/7251/12015/12016,10.0,species,Lanius ludovicianus,True


In [83]:
new_df = pd.merge( obs_taxon,processed_photos, on= "observation_uuid" )
new_df.to_csv('/Volumes/Ramirez/inaturalist-open-data/processed/new_df.csv',index=False)


In [84]:
new_df.head()

,observation_uuid,observer_id,latitude,longitude,positional_accuracy,taxon_id,quality_grade,observed_on,ancestry,rank_level,rank,name,active,photo_uuid,photo_id,extension,processed
0,b351c847-d2c6-412d-8d17-a58158893a31,516,32.767629,-117.233902,29066.0,41478.0,research,2011-01-16,48460/1/2/355675/40151/848317/848320/848324/15...,10.0,species,Eschrichtius robustus,True,7df0719d-50c7-48e1-92e5-c712b1d4386b,21362,jpg,True
1,b351c847-d2c6-412d-8d17-a58158893a31,516,32.767629,-117.233902,29066.0,41478.0,research,2011-01-16,48460/1/2/355675/40151/848317/848320/848324/15...,10.0,species,Eschrichtius robustus,True,00883152-c9ea-4738-90e2-6b30e79b5527,21363,jpg,True
2,b351c847-d2c6-412d-8d17-a58158893a31,516,32.767629,-117.233902,29066.0,41478.0,research,2011-01-16,48460/1/2/355675/40151/848317/848320/848324/15...,10.0,species,Eschrichtius robustus,True,28c8ae54-51a4-4c56-b7da-69400039144c,21364,jpg,True
3,8ad4179c-6e49-49c4-9a99-2f3019ddb573,516,34.404714,-119.692740,28818.0,4209.0,research,2008-02-12,48460/1/2/355675/3/67563/4203/4204,10.0,species,Podiceps auritus,True,9b8022a6-5908-43d5-8e9b-3642422391ea,21419,jpg,True
4,79a60b82-6f37-4eb2-b17c-4a628f9314af,516,-43.803666,172.968262,27411.0,4848.0,research,2007-06-17,48460/1/2/355675/3/67561/71325/4837,10.0,species,Haematopus unicolor,True,71b5bbcb-4f8b-4f71-8cd8-700a27b9a0b1,21559,jpg,True


In [85]:
print(new_df.isin([46017,129411,45933,45994,1431495]))

       observation_uuid  observer_id  latitude  longitude  \
0                 False        False     False      False   
1                 False        False     False      False   
2                 False        False     False      False   
3                 False        False     False      False   
4                 False        False     False      False   
...                 ...          ...       ...        ...   
11767             False        False     False      False   
11768             False        False     False      False   
11769             False        False     False      False   
11770             False        False     False      False   
11771             False        False     False      False   

       positional_accuracy  taxon_id  quality_grade  observed_on  ancestry  \
0                    False     False          False        False     False   
1                    False     False          False        False     False   
2                    False     Fa